# CT Scout Centering Notebook
This notebook mirrors the logic from the `CENTERING_CT_SCOUT` script for interactive running. Update input/output paths for your environment before running.

In [ ]:
# Section 1: Install and Import Dependencies
# If packages are missing, uncomment and run the pip installs below
# Note: In some environments, you may need to restart the kernel after install
#
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install pydicom scikit-image nibabel opencv-python matplotlib pandas scipy

import os
import copy
import glob
import warnings
from math import *
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pydicom
import cv2
import nibabel as nib
from skimage import filters, measure, morphology
from skimage.morphology import ball, binary_closing
from skimage.measure import label, regionprops
from matplotlib.pyplot import figure
from functools import reduce
from IPython.display import clear_output
from scipy.linalg import norm
import scipy.ndimage
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [ ]:
# Config: dataset, export, and processing toggles
DATASET_PATH = r"C:\\Users\\GRUNT\\Desktop\\centering\\centering\\SCOUT\\combineddfcibwhscouts"
EXPORT_DIR = r"C:\\Users\\GRUNT\\Desktop"

# Segmentation and logic parameters
THRESHOLD_HU = 100
FILL_LUNG_STRUCTURES = True

# Orientation detection tolerance (kept unused to preserve original logic)
ORIENTATION_EPSILON = 0.0

# Reporting options
REPORT_NAN_TO_CSV = True

In [ ]:
# Section 2: Configure Data Paths and Global Settings
# Update this to your dataset location
path = DATASET_PATH  # folder containing patient subfolders

# Optional parameters (kept for backward compatibility but sourced from config)
threshold_hu = THRESHOLD_HU
fill_lung_structures_default = FILL_LUNG_STRUCTURES

# Precompute fullpath list if needed (used only for filenames in the original script)
fullpath = [(path + '/' + s) for s in os.listdir(path)] if os.path.isdir(path) else []

# Initialize counters and DataFrame
lowest_dirs = []
file_count = []

df_columns = [
    "image_num_in_series", "whole_filename", "filename", "image_center",
    "x_image_center", "y_image_center", "AP_center", "x_center_coordinate_AP",
    "y_center_coordinate_AP", "lat_center", "x_center_coordinate_lat",
    "y_center_coordinate_lat", "displacement", "center_calculated_by_hand",
    "TOTAL_horiz_disp_AP,vert_disp_lat"
]

df = pd.DataFrame(columns=df_columns)

totalcounter = 0
warnings.filterwarnings("ignore")

In [ ]:
# Section 3: Define DICOM Loading and HU Conversion Helpers

def load_scan(scan_path: str):
    # Original script uses Windows-style concatenation; here we rely on os.path.join
    slices = [pydicom.dcmread(os.path.join(scan_path, s)) for s in os.listdir(scan_path)]
    # Filter to ensure SliceLocation exists (as in original)
    slices = [s for s in slices if 'SliceLocation' in s]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except Exception:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices


def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    image[image == -2000] = 0

    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope

    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)

    image += np.int16(intercept)

    return np.array(image, dtype=np.int16)

In [ ]:
# Helper: best-effort filename/identifier capture preserving current behavior

def resolve_filename(subdir: str, i: int, counter: int):
    # Original behavior: prefer fullpath[i] and fullpath[counter]
    fname_i = os.path.basename(os.path.abspath(fullpath[i])) if i < len(fullpath) else os.path.basename(subdir)
    wfname_i = os.path.abspath(fullpath[i]) if i < len(fullpath) else os.path.abspath(subdir)

    fname_counter = os.path.basename(os.path.abspath(fullpath[counter])) if counter < len(fullpath) else os.path.basename(subdir)
    wfname_counter = os.path.abspath(fullpath[counter]) if counter < len(fullpath) else os.path.abspath(subdir)

    return (fname_i, wfname_i, fname_counter, wfname_counter)

In [ ]:
# Section 4: Define Lung Segmentation Utilities

def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)
    counts = counts[vals != bg]
    vals = vals[vals != bg]
    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None


def segment_lung_mask(image, fill_lung_structures=True):
    # Binary with 1 and 2, 0 is background
    binary_image = np.array(image >= 100, dtype=np.int8) + 1  # threshold 100 HU as in script
    labels = measure.label(binary_image)

    # Fill the air around the person
    background_label = labels[0, 0, 0]

    # Corner pixel is air pixel
    binary_image[background_label == labels] = 1

    if fill_lung_structures:
        # For every slice determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)

            if l_max is not None:  # This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    binary_image -= 1
    binary_image = 1 - binary_image

    # Remove other air pockets inside body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None:
        binary_image[labels != l_max] = 0

    return binary_image

In [ ]:
# Section 5: Discover Patient Series and Initialize Results DataFrame

lowest_dirs = []
file_count = []

if os.path.isdir(path):
    for root, dirs, files in os.walk(path):
        if not dirs:
            lowest_dirs.append(root)

    for subdir in lowest_dirs:
        file_count.append(sum(len(files) for _, _, files in os.walk(subdir)))
    _ = np.array(file_count)
    length = np.sum(file_count) if len(file_count) else 0

# Create DataFrame with required columns (already created above but re-ensuring)
if df.empty and df.columns.tolist() != df_columns:
    df = pd.DataFrame(columns=df_columns)

print(f"Found {len(lowest_dirs)} leaf directories. Total files across leaves: {sum(file_count) if file_count else 0}")

In [ ]:
# Section 6: Per-Patient Processing: Segment, Compute Centers, Track Displacements

# Track any NaN centers encountered for reporting
nan_records = []

# Accumulate rows then concat for performance
rows_buffer = []

totalcounter = 0

for subdir in lowest_dirs:
    # Reset displacements per patient/series to avoid carryover if one orientation is missing
    xdisplacement = np.nan
    ydisplacement = np.nan

    patient_dicom = load_scan(subdir)
    patient_pixels = get_pixels_hu(patient_dicom)

    # get masks
    segmented_lungs = segment_lung_mask(patient_pixels, fill_lung_structures=False)
    segmented_lungs_fill = segment_lung_mask(patient_pixels, fill_lung_structures=True)

    # isolate lung from chest
    copied_pixels = copy.deepcopy(patient_pixels)
    for i, mask in enumerate(segmented_lungs_fill):
        get_high_vals = mask == 0
        copied_pixels[i][get_high_vals] = 0
    seg_lung_pixels = copied_pixels

    counter = 0
    for i in range(2):
        aplatposition = patient_dicom[i].ImagePositionPatient
        fname_i, wfname_i, fname_counter, wfname_counter = resolve_filename(subdir, i, counter)

        if aplatposition[1] == 0:  # AP orientation check
            imagenum = counter + 1
            cols = patient_dicom[i].Columns
            rows = patient_dicom[i].Rows

            image_center = np.array([int(cols / 2), int(rows / 2)])

            centroid = np.argwhere(seg_lung_pixels[i] == 0)
            centroidfilter = np.where((centroid[:, 0] == image_center[1]))
            filteredcentroid = centroid[centroidfilter]
            if filteredcentroid.size:
                xcalculated = int(np.mean(filteredcentroid[:, 1]))
            else:
                xcalculated = np.nan
                nan_records.append({
                    'subdir': subdir,
                    'image_num_in_series': i,
                    'orientation': 'AP'
                })
            patient_center = np.array([xcalculated, image_center[1]])

            displacement = patient_center - image_center
            xdisplacement = displacement[0]
            rows_buffer.append({
                'image_num_in_series': i,
                'whole_filename': subdir,
                'filename': fname_i,
                'image_center': image_center,
                'x_image_center': image_center[0],
                'y_image_center': image_center[1],
                'AP_center': patient_center,
                'x_center_coordinate_AP': xcalculated,
                'y_center_coordinate_AP': image_center[1],
                'displacement': displacement
            })

        elif aplatposition[0] == 0:  # LAT orientation check
            imagenum = counter + 1
            cols = patient_dicom[i].Columns
            rows = patient_dicom[i].Rows

            image_center = np.array([int(cols / 2), int(rows / 2)])

            centroid = np.argwhere(seg_lung_pixels[i] == 0)
            centroidfilter = np.where((centroid[:, 0] == image_center[1]))
            filteredcentroid = centroid[centroidfilter]
            if filteredcentroid.size:
                ycalculated = int(np.mean(filteredcentroid[:, 1]))
            else:
                ycalculated = np.nan
                nan_records.append({
                    'subdir': subdir,
                    'image_num_in_series': i,
                    'orientation': 'LAT'
                })
            patient_center = np.array([ycalculated, image_center[1]])

            displacement = patient_center - image_center
            ydisplacement = displacement[0]

            rows_buffer.append({
                'image_num_in_series': i,
                'whole_filename': subdir,
                'filename': fname_i,
                'image_center': image_center,
                'x_image_center': image_center[0],
                'y_image_center': image_center[1],
                'lat_center': patient_center,
                'x_center_coordinate_lat': ycalculated,
                'y_center_coordinate_lat': image_center[1],
                'displacement': displacement
            })

        counter += 1
        totalcounter += 1
        clear_output(wait=True)
        print("current file:", subdir)
        print("number of files analyzed:", totalcounter)
        warnings.filterwarnings("ignore")

    totaldisplacement = np.array([xdisplacement, ydisplacement])
    rows_buffer.append({"TOTAL_horiz_disp_AP,vert_disp_lat": totaldisplacement})

# Materialize buffer into DataFrame with concat
if rows_buffer:
    df = pd.concat([df, pd.DataFrame(rows_buffer)], ignore_index=True)

# Print a consolidated warning for NaNs encountered
if nan_records:
    summary = [f"{r['subdir']} (i={r['image_num_in_series']}, {r['orientation']})" for r in nan_records]
    print("WARNING: NaN patient centers encountered for:")
    for s in summary:
        print(" -", s)

print("Per-patient processing complete.")

In [ ]:
# Section 7: Export Aggregated Results to CSV
# Adjust output directory as needed
output_dir = EXPORT_DIR
try:
    os.makedirs(output_dir, exist_ok=True)
    out_csv = os.path.join(output_dir, "test.csv")
    df.to_csv(out_csv, index=False)
    print(f"Dataset exported to {out_csv}.")

    # Optional: export NaN summary
    if REPORT_NAN_TO_CSV and 'nan_records' in globals() and nan_records:
        nan_df = pd.DataFrame(nan_records)
        nan_csv = os.path.join(output_dir, "test_nan_summary.csv")
        nan_df.to_csv(nan_csv, index=False)
        print(f"NaN summary exported to {nan_csv}.")
except Exception as e:
    print("Failed to export dataset:", e)
print("done.")

In [ ]:
# Section 8: Load One Test Image by Index and Orientation

filenum = 9
APorLAT = 0  # 0 for AP, 1 for LAT (for most of the images)

newpath = os.path.join(path, str(filenum), "IMAGES")
if os.path.isdir(newpath):
    files_in_newpath = os.listdir(newpath)
    if files_in_newpath:
        print("filename is:", files_in_newpath[APorLAT])
else:
    print("Warning: test folder not found:", newpath)

patient_dicom = load_scan(newpath)
patient_pixels = get_pixels_hu(patient_dicom)

In [ ]:
# Section 9: Segment Test Image and Compute Centers

# get masks
segmented_lungs = segment_lung_mask(patient_pixels, fill_lung_structures=False)
segmented_lungs_fill = segment_lung_mask(patient_pixels, fill_lung_structures=True)

# isolate lung from chest
copied_pixels = copy.deepcopy(patient_pixels)
for i, mask in enumerate(segmented_lungs_fill):
    get_high_vals = mask == 0
    copied_pixels[i][get_high_vals] = 0
seg_lung_pixels = copied_pixels

cols = patient_dicom[APorLAT].Columns
rows = patient_dicom[APorLAT].Rows
image_center = np.array([int(cols / 2), int(rows / 2)])

print("image center is:", image_center)

centroid = np.argwhere(seg_lung_pixels[APorLAT] == 0)
centroidfilter = np.where((centroid[:, 0] == image_center[1]))
filteredcentroid = centroid[centroidfilter]
xcalculated = int(np.mean(filteredcentroid[:, 1])) if filteredcentroid.size else image_center[0]

patient_center = np.array([xcalculated, image_center[1]])
print("patient center is:", patient_center)

In [ ]:
# Section 10: Visualize Single Slice with Centers

figure(figsize=(5, 5), dpi=250)
plt.title("Slice With Centers Marked", size=10)
plt.xlabel("mm", size=8)
plt.ylabel("mm", size=8)

plt.imshow(patient_pixels[APorLAT], cmap=plt.cm.bone)
plt.scatter(patient_center[0], patient_center[1], marker="x", color="red", s=10)
plt.scatter(image_center[0], image_center[1], marker="o", color="blue", s=10)
A = "Region Center"
B = "FOV Center"
plt.legend([A, B], prop={'size': 6}, loc='upper right', bbox_to_anchor=(1.7, 1))
plt.show()

In [ ]:
# Section 11: Batch Visualization Across Multiple Studies and Views

for filenum in [x for x in range(39, 41)]:  # adjust ranges as needed
    for APorLAT in [0, 1]:
        newpath = os.path.join(path, str(filenum), "IMAGES")
        print("path to file is:", newpath[48:-6] if len(newpath) >= 54 else newpath)

        patient_dicom = load_scan(newpath)
        patient_pixels = get_pixels_hu(patient_dicom)

        # get masks
        segmented_lungs = segment_lung_mask(patient_pixels, fill_lung_structures=False)
        segmented_lungs_fill = segment_lung_mask(patient_pixels, fill_lung_structures=True)

        # isolate lung from chest
        copied_pixels = copy.deepcopy(patient_pixels)
        for i, mask in enumerate(segmented_lungs_fill):
            get_high_vals = mask == 0
            copied_pixels[i][get_high_vals] = 0
        seg_lung_pixels = copied_pixels

        cols = patient_dicom[APorLAT].Columns
        rows = patient_dicom[APorLAT].Rows
        image_center = np.array([int(cols / 2), int(rows / 2)])

        print("image center is:", image_center)

        centroid = np.argwhere(seg_lung_pixels[APorLAT] == 0)
        centroidfilter = np.where((centroid[:, 0] == image_center[1]))
        filteredcentroid = centroid[centroidfilter]
        xcalculated = int(np.mean(filteredcentroid[:, 1])) if filteredcentroid.size else image_center[0]

        patient_center = np.array([xcalculated, image_center[1]])
        print("patient center is:", patient_center)

        A = "Region Center"
        B = "FOV Center"

        f, ax = plt.subplots(1, 2, figsize=(6, 4), dpi=200)
        ax[0].imshow(patient_pixels[APorLAT], cmap=plt.cm.bone)
        ax[0].scatter(patient_center[0], patient_center[1], marker="x", color="red", s=10)
        ax[0].scatter(image_center[0], image_center[1], marker="o", color="cornflowerblue", s=10)
        ax[0].axis(True)
        ax[0].set_xlabel("mm", size=8)
        ax[0].set_ylabel("mm", size=8)
        ax[0].legend([A, B], prop={'size': 6}, loc='upper right', bbox_to_anchor=(1.95, 1))
        ax[0].set_title('Original Slice', size=10)

        ax[1].imshow(seg_lung_pixels[APorLAT], cmap=plt.cm.bone)
        ax[1].scatter(patient_center[0], patient_center[1], marker="x", color="red", s=10)
        ax[1].scatter(image_center[0], image_center[1], marker="o", color="slateblue", s=10)
        ax[1].axis(True)
        ax[1].set_title('Segmented Slice')
        ax[1].set_xlabel("mm", size=8)
        ax[1].set_ylabel("mm", size=8)
        ax[1].legend([A, B], prop={'size': 6}, loc='upper right', bbox_to_anchor=(1.95, 1))

        warnings.filterwarnings("ignore")
        plt.show()

print("Batch visualization complete.")